<a href="https://colab.research.google.com/github/Quanhcmus/Image_classification_CIFAR10_with_VGG11/blob/main/CNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import keras
from keras.datasets import cifar10
from keras.models import Sequential
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.layers import Conv2D, Flatten,AveragePooling2D
from sklearn.model_selection import train_test_split

num_class=10
batch_size=32
epoch=50
data_augmentation = True

(X,y),(X_test,y_test)=cifar10.load_data()
X_train,X_valid,y_train,y_valid=train_test_split(X,y,test_size=0.25,random_state=42)
print('x_train shape:', X_train.shape)
print(X_train.shape[0], 'train samples')
print(X_valid.shape[0], 'valid samples')
print(X_test.shape[0], 'test samples')

170498071/170498071 [==============================] - 2s 0us/step
x_train shape: (37500, 32, 32, 3)
37500 train samples
12500 valid samples
10000 test samples


In [2]:
# Convert class vectors to binary class matrices.
y_train=keras.utils.to_categorical(y_train)
y_test=keras.utils.to_categorical(y_test)

In [3]:
# tất cả các pixel của ảnh đều nằm trong khoảnh [0;255] nên chia cho 255 để tất cả đều nằm trong khoảng [0;1]
X_train=X_train.astype('float32')
X_test=X_test.astype('float32')
X_train/=255
X_test/=255

In [4]:
# Letnet5

model= Sequential()

model.add(Conv2D(6,(5,5),input_shape=X_train.shape[1:]))
model.add(Activation('relu'))
model.add(AveragePooling2D())

model.add(Conv2D(16,(5,5)))
model.add(Activation('relu'))
model.add(AveragePooling2D())
model.add(Flatten())

model.add(Dense(120))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(84))
model.add(Activation('relu'))
model.add(Dropout(0.25))

model.add(Dense(10))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.Adam(lr=0.0001)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])
model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 6)         456       
                                                                 
 activation (Activation)     (None, 28, 28, 6)         0         
                                                                 
 average_pooling2d (AverageP  (None, 14, 14, 6)        0         
 ooling2D)                                                       
                                                                 
 conv2d_1 (Conv2D)           (None, 10, 10, 16)        2416      
                                                                 
 activation_1 (Activation)   (None, 10, 10, 16)        0         
                                                                 
 average_pooling2d_1 (Averag  (None, 5, 5, 16)         0         
 ePooling2D)                                            

/usr/local/lib/python3.10/dist-packages/keras/optimizers/legacy/adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [5]:
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph2', histogram_freq=0, write_graph=True, write_images=True)

In [6]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(X_train, y_train,
              batch_size=batch_size,
              epochs=epoch,
              validation_data=(X_test, y_test),
              shuffle=True, callbacks=[tbCallBack])
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    '''
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images
    '''
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=10,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.2,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.2,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images


    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(X_train)

    # Fit the model on the batches generated by datagen.flow().
    model.fit_generator(datagen.flow(X_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=X_train.shape[0] // batch_size,
                        epochs=epoch,
                        validation_data=(X_test, y_test), callbacks=[tbCallBack])


Using real-time data augmentation.
Epoch 1/50


<ipython-input-6-fafaa43913b4>:42: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(datagen.flow(X_train, y_train,


1171/1171 [==============================] - 37s 24ms/step - loss: 2.1817 - accuracy: 0.1834 - val_loss: 1.9871 - val_accuracy: 0.2796
Epoch 2/50
1171/1171 [==============================] - 27s 23ms/step - loss: 2.0116 - accuracy: 0.2558 - val_loss: 1.8424 - val_accuracy: 0.3395
Epoch 3/50
1171/1171 [==============================] - 28s 24ms/step - loss: 1.9131 - accuracy: 0.2943 - val_loss: 1.7124 - val_accuracy: 0.3875
Epoch 4/50
1171/1171 [==============================] - 27s 23ms/step - loss: 1.8391 - accuracy: 0.3201 - val_loss: 1.6586 - val_accuracy: 0.4048
Epoch 5/50
1171/1171 [==============================] - 28s 24ms/step - loss: 1.7820 - accuracy: 0.3439 - val_loss: 1.5952 - val_accuracy: 0.4255
Epoch 6/50
1171/1171 [==============================] - 30s 26ms/step - loss: 1.7529 - accuracy: 0.3522 - val_loss: 1.5616 - val_accuracy: 0.4367
Epoch 7/50
1171/1171 [==============================] - 28s 24ms/step - loss: 1.7271 - accuracy: 0.3632 - val_loss: 1.5556 - val_accura

In [7]:
scores=model.evaluate(X_test,y_test,verbose=1)
print("Test Loss",scores[0])
print("Test Accuracy",scores[1])

313/313 [==============================] - 1s 3ms/step - loss: 1.2638 - accuracy: 0.5475
Test Loss 1.2637794017791748
Test Accuracy 0.5475000143051147
